### Основные типы графовых моделей

#### Графовые нейронные сети (GNN — Graph Neural Networks)

Это общий класс моделей, способных обрабатывать графы. Основная идея — агрегация информации от соседей для обновления представления узла.

Цель GNN — научиться представлять каждый узел (или весь граф) в виде вектора (эмбеддинга), который учитывает:
- признаки самого узла;
- признаки его соседей;
- структуру графа.

Эти эмбеддинги затем используются для задач:
- классификации узлов (например, определить, является ли пользователь ботом);
- классификации графов (например, токсична ли молекула);
- предсказания рёбер (например, предсказать, подружатся ли два пользователя).

**Основная идея: Message Passing (передача сообщений)**

Каждый узел обменивается информацией со своими соседями. Этот процесс повторяется несколько раз (слоёв), и на каждом шаге узел обновляет своё представление.

Алгоритм одного слоя GNN:
1. Агрегация: собрать представления соседей;
2. Обновление: обновить своё представление на основе собственного и агрегированного от соседей.
Формально:
$$\mathbf{h}_v^{(k)} = \sigma \left( \mathbf{W}^{(k)} \cdot \text{AGGREGATE}^{(k)} \left( \left\{ \mathbf{h}_u^{(k-1)} \mid u \in \mathcal{N}(v) \right\} \cup \left\{ \mathbf{h}_v^{(k-1)} \right\} \right) \right)$$
где:
- $\mathbf{h}_v^{(k)}$ - эмбеддинг узла *v* на слое *k*;
- $\mathcal{N}(v)$ - соседи узла *v*;
- $\text{AGGREGATE}$ - функция агрегации (среднее, сумма, максимум и т.д.);
- $\mathbf{W}^{(k)}$ - обучаемая матрица весов;
- $\sigma$ - нелинейная функция активации (например, ReLU).

**Пример: как работает GNN (визуализация в тексте)**

Рассмотрим простой граф из 4х узлов:

<img src="src/graph_1.svg" width="300" height="200">

- Узлы: A, B, C, D
- Рёбра: A–B, A–C, B–D, C–D

Допустим, у каждого узла есть начальный признак:
- A: [1, 0]
- B: [0, 1]
- C: [1, 1]
- D: [0, 0]

1. Шаг 0 (начальные эмбеддинги)
2. Шаг 1 (первый слой GNN — агрегация соседей через среднее):
   
   - Для A: соседи = B, C → среднее = ([0,1] + [1,1]) / 2 = [0.5, 1.0]
  
        Новое представление A = MLP([1,0] + [0.5,1.0]) → например, [0.8, 0.9]
   - Для D: соседи = B, C → среднее = ([0,1] + [1,1]) / 2 = [0.5, 1.0]
        
        Новое D = MLP([0,0] + [0.5,1.0]) → например, [0.4, 0.7]

Где применяются GNN:
- Химия: Предсказание свойств молекул (узлы = атомы, рёбра = связи);
- Соцсети: Обнаружение сообществ, рекомендации;
- Биоинформатика: Протеин-протеин взаимодействия;
- Рекомендации: Пользователь–товар граф
- Финансы: Обнаружение мошенничества в транзакциях

#### Графовые автокодировщики (Graph Autoencoders, GAE)

Графовый автокодировщик (GAE) — это нейросетевая модель, которая обучается без учителя (unsupervised) для получения низкоразмерных векторных представлений (эмбеддингов) узлов графа, сохраняя при этом его структуру.

**Цель**: Научиться кодировать граф так, чтобы по эмбеддингам можно было восстановить связи (рёбра) между узлами.

Это особенно полезно, когда:
- Нет меток у узлов (например, в соцсетях);
- Нужно предсказать, появится ли связь между двумя узлами (link prediction);
- Требуется сжатое представление графа для последующих задач (кластеризация, визуализация и т.д.).

**Архитектура GAE**

GAE состоит из двух частей:
1. **Энкодер (Encoder)**

Преобразует каждый узел $v_i$ в скрытое представление (эмбеддинг) $z_i \in \mathbb{R}^d, \text{ где } d \ll N$ (N — число узлов).

Чаще всего энкодер — это GNN (например, GCN):
$$Z=GCN(X,A)$$
- *X* - матрица признаков узлов (если нет признаков, можно использовать единичную матрицу *I*)
- *A* - матрица смежности графа
- *Z* - матрица эмбеддингов размером $N×d$

2. **Декодер (Decoder)**

Восстанавливает (предсказывает) матрицу смежности из $\hat{A}$ эмбеддингов Z.

Самый простой декодер — внутреннее произведение:
$$\hat{A}_{ij} = \sigma(z_i^{T}  z_j)$$
где $\sigma$ - сигмоида (дает вероятность существования ребра между *i* и *j*).

**Функция потерь:**

Обычно используется бинарная кросс-энтропия между истинной матрицей смежности *A* и предсказанной $\hat{A}$:
$$\mathcal{L} = - \sum_{i,j} \left[ A_{ij} \log \hat{A}_{ij} + (1 - A_{ij}) \log(1 - \hat{A}_{ij}) \right]$$
На практике часто применяют взвешенную или сэмплированную версию, чтобы сбалансировать редкие рёбра (графы обычно разрежены).

**Пример: GAE на простом графе:**

Рассмотрим маленький неориентированный граф из 4x узлов:

- Узлы: A, B, C, D
- Рёбра: A–B, B–C, C–D

<img src="src/graph_2.svg" width="200" height="50">

**Шаг 1: Исходные данные**
- Матрица смежности A :
```text
        [[0, 1, 0, 0],
         [1, 0, 1, 0],
         [0, 1, 0, 1],
         [0, 0, 1, 0]]
```
- Признаки узлов $X = I_4$ (единичная матрица, т.к. признаков нет).

**Шаг 2: Энкодер (GCN)**

Пусть GCN даёт эмбеддинги размерности 2:

| Узел | Эмбеддинг $Z_1$ |
|---|---|
|A|[0.8, -0.3]|
|B|[0.6, 0.1]|
|C|[-0.5, 0.7]|
|D|[-0.9, 0.4]|

**Шаг 3: Декодер (внутреннее произведение + сигмоида)**

Вычислим $\hat{A}_{AB} = \sigma([0.8, -0.3]·[0.6, 0.1]) = \sigma(0.48 - 0.3) = \sigma(0.45) ≈ 0.61$

Аналогично для всех пар → получаем $\hat{A}$, близкую к исходной $A$.

**Применение GAE**
- Link Prediction → Предсказать, появится ли дружба в соцсети или взаимодействие белков;
- Node Clustering → Кластеризовать узлы по эмбеддингам (например, найти сообщества в Reddit);
- Graph Visualization → Снизить размерность до 2D/3D для визуализации (аналог t-SNE, но с учётом структуры);
- Предобучение для других задач → Эмбеддинги из GAE можно использовать как вход для классификатора (например, определить мошенника в графе транзакций).

#### Графовые трансформеры (Graph Transformers)

Graph Transformers (GT) — это адаптация классических трансформеров (известных по NLP и Vision) к неевклидовым данным, то есть к графам. В отличие от CNN или RNN, трансформеры изначально не учитывают структуру графа — они работают с последовательностями и используют self-attention, которая по умолчанию полносвязна (каждый узел "видит" все остальные).

Но в графах:
- Не все узлы связаны;
- Важна топология (расстояния, пути, сообщества);
- Информация должна распространяться локально, как в GNN.

Поэтому Graph Transformers модифицируют механизм внимания, чтобы он учитывал графовую структуру.

**Основная идея**

В классическом трансформере для последовательности токенов $x_1, ..., x_n$:
$$\text{Attention}(Q, K, V) = \text{softmax}\left( \frac{QK^T}{\sqrt{d}} \right) V$$

В Graph Transformer мы хотим, чтобы внимание не было полносвязным, а зависело от:
- Существования ребра между узлами;
- Расстояния в графе;
- Позиционной информации (например, кратчайшего пути);
- Типа узла/ребра (в гетерогенных графах).

**Ключевые компоненты Graph Transformers**
1. **Позиционные эмбеддинги (Positional Encodings)**

Так как графы не имеют естественного порядка (в отличие от текста), нужно явно закодировать геометрию графа.

Популярные подходы:
- Laplacian Eigenvectors (спектральные координаты);
- Shortest-path distances → эмбеддинги расстояний;
- Random Walk Encodings;
- Node2Vec / DeepWalk как предобученные признаки

2. **Структурированное внимание**

Вместо полносвязного внимания:
- Ограничить внимание соседями (как в GNN) → Sparse Graph Transformer;
- Взвешивать внимание по расстоянию → ближние узлы важнее;
- Использовать edge features в attention

Пример формулы с учётом рёбер:
$$\text{score}_{ij} = \frac{(W_Q h_i)^T (W_K h_j)}{\sqrt{d}} + \psi(e_{ij}) + \phi(\text{dist}(i, j))$$
где:
- $h_i$ - признак узла *i*;
- $e_{ij}$ - признак ребра между *i* и *j*;
- $\psi, \phi$ - нейросети для кодирования ребер и расстояний.

3. **Многоуровневая агрегация**

Некоторые модели (например, GraphGPS) комбинируют:
- Message Passing (GNN-стиль) — для локального контекста;
- Global Attention (Transformer-стиль) — для долгосрочных зависимостей

Это даёт лучшую выразительность и устойчивость к oversmoothing.

**Примеры известных Graph Transformer моделей**

|Модель|Год | Ключевая идея |
|---|---|---|
|Graphormer|2021 (Microsoft)|Использует расстояния, степени узлов и edge features в attention bias|
|SAN (Structure-Aware Transformer)|2022|Встраивает структуру через Laplacian PE|
|GraphGPS|2022|Гибрид GNN + Transformer|
|GTN (Graph Transformer Network)|2020|Учитывает метапути в гетерогенных графах|
|Phyloformer|2022|Для филогенетических деревьев (специфический граф)|

**Пример: как работает Graphormer на маленьком графе**

Рассмотрим простой граф из 4 узлов:
<img src="src/graph_1.svg" width="300" height="200">

- Узлы: A, B, C, D
- Рёбра: A–B, A–C, B–D, C–D

**Шаг 1: Вычисляем матрицу расстояний:**

|  |A|B|C|D|
|---|---|---|---|---|
|A|0|1|1|2|
|B|1|0|2|1|
|C|1|2|0|1|
|D|2|1|1|0|

**Шаг 2: Для каждой пары (*i*,*j*) добавляем bias в attention:**
- Если dist(i,j) = 1 → сильный положительный bias;
- Если dist(i,j) = 2 → слабый или отрицательный
- Если нет пути → очень низкий вес

**Шаг 3: Self-attention теперь "знает", что A и D не связаны напрямую, но связаны через B/C.**

В итоге эмбеддинг узла A будет учитывать не только B и C, но и косвенно D — но с меньшим весом.

**Визуализация: как меняется внимание**

Допустим, у нас есть attention-матрица до и после добавления структурного bias:

Без структуры (обычный Transformer):
```text
      A    B    C    D
A  [0.25 0.25 0.25 0.25]
B  [0.25 0.25 0.25 0.25]
C  [0.25 0.25 0.25 0.25]
D  [0.25 0.25 0.25 0.25]
```
→ Все узлы равнозначны.

**С Graph Transformer (например, Graphormer):**
```text
      A    B    C    D
A  [0.50 0.25 0.25 0.00]   ← A "видит" только B и C
B  [0.30 0.40 0.00 0.30]   ← B видит A и D
C  [0.30 0.00 0.40 0.30]   ← C видит A и D
D  [0.00 0.35 0.35 0.30]   ← D видит B и C
```
→ Внимание соответствует структуре графа.